In [40]:
# importar librerias
import time
import pandas as pd
from datetime import datetime, timedelta
from gspread_dataframe import get_as_dataframe
import re
import pandas as pd
import gspread
import time
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import unicodedata, re

In [2]:
# 0. Autenticación con tu archivo JSON
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credenciales.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
# 🧼 1. Función para limpiar nombres de columnas
def normalizar_columna(nombre):
    nombre = nombre.strip().lower()
    nombre = re.sub(r'[\n\r]+', ' ', nombre)
    nombre = re.sub(r'[^\w\s]', '', nombre)
    nombre = re.sub(r'\s+', ' ', nombre)
    return nombre.strip()

In [4]:
# 2. Archivos por año (reemplaza con tus IDs reales)
files = {
    '2023': '1dBw6sx91ogtriSMztGKkn2IkTCUCvLD77jKZNOioMNU',                   #eliminar dos primeras filas
    '2024': '1fKXbm3xWofNEozkJrZ4p6B4svWAI0ZqHzio-1MaAp8s',                   #eliminar dos primeras filas
    '2025': '1WQbRXClUxdnqudkb1Kskq2g8TlroWd5bc_IzAw2UbwU'                    #eliminar dos primeras filas
}

In [ ]:
# 3. Pre procesar el consolidado para investigar

# === PRE-CONSOLIDADO CRUDO con normalización básica (igual a tu código) ===
# Normaliza encabezados: strip + lower + reemplazo de espacios por "_"

tiempo_maximo = timedelta(minutes=10)   # límite de ejecución total
inicio = datetime.now()
crudos = []

for año, file_id in files.items():
    try:
        sh = gc.open_by_key(file_id)  # abrir el Google Sheet por su ID
    except Exception as e:
        print(f"⚠️ No se pudo abrir el archivo {file_id} ({año}): {e}")
        continue

    for hoja in sh.worksheets():
        try:
            # Chequeo de tiempo
            if datetime.now() - inicio > tiempo_maximo:
                print("⏳ Límite de tiempo alcanzado, deteniendo ejecución (crudo).")
                break

            # Lee la hoja (primera fila como encabezado) y elimina filas completamente vacías
            df = get_as_dataframe(hoja, evaluate_formulas=True, header=0).dropna(how='all')

            if df.empty:
                print(f"ℹ️ Hoja vacía (o solo filas vacías): {hoja.title} ({año})")
                time.sleep(1.0)
                continue

            # --- Normalización básica de encabezados (igual a tu lambda) ---
            df = df.loc[:, ~df.columns.duplicated()]
            df.rename(
                columns=lambda x: str(x).strip().lower().replace(" ", "_"),
                inplace=True
            )
            df = df.loc[:, ~df.columns.duplicated()]

            # Agregar contexto mínimo
            df["Año"] = año
            df["Hoja_Origen"] = hoja.title
            df["Archivo_ID"] = file_id

            crudos.append(df)
            print(f"✅ CRUDO normalizado (básico): {hoja.title} ({año}) con {len(df)} filas")

            # Pausa corta para evitar quota errors
            time.sleep(1.5)

        except Exception as e:
            print(f"⚠️ Error leyendo hoja {hoja.title} ({año}): {e}")
    else:
        # Si NO hubo break en el for de hojas, sigue con el siguiente archivo
        continue

    # Si hubo break (por tiempo), rompe el for externo también
    break

# 🔗 Concatenación cruda
if crudos:
    df_crudo_total = pd.concat(crudos, ignore_index=True)
    print(f"\n✅ Total hojas CRUDO procesadas: {len(crudos)} | Filas: {len(df_crudo_total)}")
else:
    df_crudo_total = pd.DataFrame()
    print("⚠️ No se consolidó ninguna hoja en CRUDO.")


✅ CRUDO normalizado (básico): EDERLY (2023) con 39 filas
✅ CRUDO normalizado (básico): ANDREA (2023) con 67 filas
✅ CRUDO normalizado (básico): MELANIE (2023) con 15 filas
✅ CRUDO normalizado (básico): ROSSMERY (2023) con 49 filas
✅ CRUDO normalizado (básico): MICHEL (2023) con 80 filas
✅ CRUDO normalizado (básico): MARINA (2023) con 70 filas
✅ CRUDO normalizado (básico): SHARON (2023) con 69 filas
✅ CRUDO normalizado (básico): VICTOR (2023) con 77 filas
✅ CRUDO normalizado (básico): YESSICA (2023) con 38 filas
✅ CRUDO normalizado (básico): MARJORIE (2023) con 17 filas
✅ CRUDO normalizado (básico): SHIRLEY (2023) con 26 filas
✅ CRUDO normalizado (básico): FLOR (2023) con 48 filas
✅ CRUDO normalizado (básico): ORLANDO (2023) con 16 filas
✅ CRUDO normalizado (básico): CRISTINA (2023) con 29 filas
✅ CRUDO normalizado (básico): CAMILLE (2023) con 38 filas
✅ CRUDO normalizado (básico): BENJHAMIN (2023) con 9 filas
✅ CRUDO normalizado (básico): Total (2023) con 57 filas
⚠️ No se pudo abrir e

In [ ]:
# 4. Investigar y visualizar las columnas
print(list(df_crudo_total.columns))


['n°', 'cuenta_/\nagencia\n1', 'tipo_venta', 'tipo_de_file', 'nombre_file', 'mercado', 'ejecutivo_ventas', 'número_paxs\nj', 'fecha_de_1er_contacto\n?', 'fecha_aceptac.\n?', 'fecha_del_viaje\nq', 'duracion_del_viaje\nº', 'destinos\nþ', 'nombres\napellidos\n!', 'pais_origen', 'ciudad', 'perfil_del_viaje', 'n°_\ntelefono\n)', 'e-mail\n*@', 'rango\nedad', 'precio_de_venta_unitario_(us$)', 'precio_de_venta_total\n(us$)', '%_comisión_de_venta', 'tasa_de_cambio\n(us$_a_€)', 'precio_de_venta_total\n(€)', '1er_pago\n(us$)', 'fecha_del_1er_pago', '1er_pago_al_cambio\n(us$_a_€)', 'canal_de_1er_pago', 'n°_operación_financiera', '2do_pago\n(us$)', 'fecha_del_pago_del_saldo', 'tasa_de_cambio\n(us$_a_€).1', '2do_pago_al_cambio\n(us$_a_€)', 'canal_de_2do_pago', 'n°_operación_financiera_.1', 'status\npago/final\n($,_€_ò_s/)', 'solicitud_de_postventa', 'monto_de\nreembolso\n(us$)', 'canal_de_reembolso', 'comentarios_adicionales', 'Año', 'Hoja_Origen', 'Archivo_ID', 'campaña', 'medio_de_contacto', '1er_

In [ ]:
# === Proceso de consolidar y extraer el df_total solo con las columans necesarias ===

# 5. Diccionario de mapeo (versión resumida; ajusta si lo necesitas)
mapeo_columnas_manual = {
    '% comisión de venta': 'porc_comision',
    'campaña': 'campaña',
    'canal': 'canal',
    'ciudad': 'ciudad',
    'cuenta / agencia 1': 'canal',
    'destinos': 'destinos',
    'destinos þ': 'destinos',
    'duracion del viaje': 'duracion',
    'duracion del viaje º': 'duracion',
    'e-mail *@': 'email',
    'edad': 'edad',
    'ejecutivo ventas': 'ejecutivo',
    'email': 'email',
    'fecha aceptac.': 'fecha_aceptacion',
    'fecha aceptac. ?': 'fecha_aceptacion',
    'fecha de 1er /contacto ?': 'fecha_contacto',
    'fecha de 1er contacto': 'fecha_contacto',
    'fecha de 1er contacto ?': 'fecha_contacto',
    'fecha de aceptacion del viaje ?': 'fecha_aceptacion',
    'fecha de venta. ?': 'fecha_aceptacion',
    'fecha del viaje': 'fecha_viaje',
    'fecha del viaje q': 'fecha_viaje',
    'medio de contacto': 'medio_contacto',
    'mercado': 'mercado',
    'nombre file': 'file',
    'nombres apellidos !': 'nombre_completo',
    'nro': 'nro',
    'n°': 'nro',
    'n°  telefono )': 'telefono',
    'n° paxs j': 'nro_paxs',
    'n° telefono': 'telefono',
    'número pax': 'nro_paxs',
    'número paxs j': 'nro_paxs',
    'pais origen': 'pais_origen',
    'perfil de viaje': 'perfil_viaje',
    'perfil del viaje': 'perfil_viaje',
    'precio de venta total (us$)': 'precio_total_usd',
    'precio de venta total (€)': 'precio_total_eur',
    'precio de venta unitario (us$)': 'precio_unitario',
    'rango edad': 'edad',
    'referido nuevo': 'referido_nuevo',
    'telefono': 'telefono',
    'tipo de file': 'tipo_file',
    'tipo venta': 'referido_nuevo'
}
# 📌 6. Columnas finales esperadas
columnas_finales = [
    'nombres\napellidos\n!','pais_origen','ciudad','n°_\ntelefono\n)','e-mail\n*@', 'fecha_aceptac.\n?','fecha_de_aceptacion_del_viaje\n?','fecha_de_venta.\n?','precio_de_venta_total\n(us$)','precio_de_venta_total\n(€)'
]
# 7. Parámetros de control
tiempo_maximo = timedelta(minutes=10)  # límite de ejecución total
inicio = datetime.now()
consolidados = []

for año, file_id in files.items():
    sh = gc.open_by_key(file_id)                   # abrir el archivo 
    for hoja in sh.worksheets():
        try:
            # Verifica límite de tiempo
            if datetime.now() - inicio > tiempo_maximo:
                print("⏳ Límite de tiempo alcanzado, deteniendo ejecución.")
                break

            # Lee la hoja (primera fila como encabezado) y elimina filas totalmente vacías
            df = get_as_dataframe(hoja, evaluate_formulas=True, header=0).dropna(how='all')

            # Eliminar columnas duplicadas antes y después de renombrar
            df = df.loc[:, ~df.columns.duplicated()]
            df.rename(
                columns=lambda x: mapeo_columnas_manual.get(
                    str(x).strip(),
                    str(x).strip().lower().replace(" ", "_")
                ),
                inplace=True
            )
            df = df.loc[:, ~df.columns.duplicated()]

            # Añadir columnas contextuales
            df['Año'] = año
            
            df['Hoja_Origen'] = hoja.title

            # ✅ Filtrar y reordenar columnas
            columnas_presentes = [col for col in columnas_finales if col in df.columns]
            columnas_ordenadas = columnas_presentes + ['Año', 'Hoja_Origen']
            df = df.reindex(columns=columnas_ordenadas)

            consolidados.append(df)
            print(f"✅ Consolidada hoja: {hoja.title} ({año})")

            time.sleep(2.5)  # pausa para evitar quota errors

        except Exception as e:
            print(f"⚠️ Error en hoja {hoja.title}: {e}")
    else:
        continue
    break  # rompe si se alcanzó el tiempo

# 🔗 Combina todas las hojas consolidadas
if consolidados:
    df_total = pd.concat(consolidados, ignore_index=True)
    print(f"\n✅ Total hojas procesadas: {len(consolidados)}")
else:
    print("⚠️ No se consolidó ninguna hoja.")




✅ Consolidada hoja: EDERLY (2023)
✅ Consolidada hoja: ANDREA (2023)
✅ Consolidada hoja: MELANIE (2023)
✅ Consolidada hoja: ROSSMERY (2023)
✅ Consolidada hoja: MICHEL (2023)
✅ Consolidada hoja: MARINA (2023)
✅ Consolidada hoja: SHARON (2023)
✅ Consolidada hoja: VICTOR (2023)
✅ Consolidada hoja: YESSICA (2023)
✅ Consolidada hoja: MARJORIE (2023)
✅ Consolidada hoja: SHIRLEY (2023)
✅ Consolidada hoja: FLOR (2023)
✅ Consolidada hoja: ORLANDO (2023)
✅ Consolidada hoja: CRISTINA (2023)
✅ Consolidada hoja: CAMILLE (2023)
✅ Consolidada hoja: BENJHAMIN (2023)
✅ Consolidada hoja: Total (2023)
✅ Consolidada hoja: Andrea (2024)
✅ Consolidada hoja: Arturo (2024)
✅ Consolidada hoja: Rony (2024)
✅ Consolidada hoja: Dahir (2024)
✅ Consolidada hoja: Estrella (2024)
✅ Consolidada hoja: Jennifer (2024)
✅ Consolidada hoja: Marina (2024)
✅ Consolidada hoja: Rossmery (2024)
✅ Consolidada hoja: Sheri (2024)
✅ Consolidada hoja: Cristina (2024)
✅ Consolidada hoja: Vanessa (2024)
✅ Consolidada hoja: Morgane (202

In [ ]:

df_total

,nombres\napellidos\n!,pais_origen,ciudad,n°_\ntelefono\n),e-mail\n*@,fecha_aceptac.\n?,precio_de_venta_total\n(us$),precio_de_venta_total\n(€),Año,Hoja_Origen,fecha_de_aceptacion_del_viaje\n?,fecha_de_venta.\n?
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,EDERLY,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,EDERLY,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,EDERLY,NaN,NaN
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,06/03/23,5050.0,4848,2023,EDERLY,NaN,NaN
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,30/03/23,16640.0,15974.4,2023,EDERLY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Yashira,NaN,NaN
2995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Yashira,NaN,NaN
2996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Yashira,NaN,NaN
2997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Yashira,NaN,NaN


In [ ]:
# N = 5
# for anio, file_id in files.items():
#     sh = gc.open_by_key(file_id)
#     print(f"\n=== {anio} :: {sh.title} ===")
#     for ws in sh.worksheets():
#         try:
#             vals = ws.get_all_values()[:N]
#             print(f"--- Hoja: {ws.title} (primeras {N} filas crudas) ---")
#             for i, fila in enumerate(vals, start=1):
#                 print(f"Fila {i}: {fila}")
#         except Exception as e:
#             print(f"  ⚠️ {ws.title}: {e}")



=== 2023 :: REPORTE DE NUEVAS VENTAS 2023.xlsm ===
--- Hoja: EDERLY (primeras 5 filas crudas) ---
Fila 1: ['N°', 'CUENTA /\nAGENCIA\n1', 'TIPO VENTA', 'TIPO DE FILE', 'NOMBRE FILE', 'MERCADO', 'EJECUTIVO VENTAS', 'NÚMERO PAXS\nJ', 'FECHA DE 1ER CONTACTO\n?', 'FECHA ACEPTAC.\n?', 'FECHA DEL VIAJE\nQ', 'DURACION DEL VIAJE\nº', 'DESTINOS\nþ', 'NOMBRES\nAPELLIDOS\n!', 'PAIS ORIGEN', 'CIUDAD', 'PERFIL DEL VIAJE', 'N° \nTELEFONO\n)', 'E-mail\n*@', 'RANGO\nEDAD', 'PRECIO DE VENTA UNITARIO (US$)', 'PRECIO DE VENTA TOTAL\n(US$)', '% COMISIÓN DE VENTA', 'TASA DE CAMBIO\n(US$ a €)', 'PRECIO DE VENTA TOTAL\n(€)', '1er PAGO\n(US$)', 'FECHA DEL 1er PAGO', '1er PAGO AL CAMBIO\n(US$ a €)', 'CANAL DE 1er PAGO', 'N° OPERACIÓN FINANCIERA ', '2do PAGO\n(US$)', 'FECHA DEL PAGO DEL SALDO', 'TASA DE CAMBIO\n(US$ a €)', '2do PAGO AL CAMBIO\n(US$ a €)', 'CANAL DE 2do PAGO', 'N° OPERACIÓN FINANCIERA ', 'STATUS\nPAGO/FINAL\n($, € ò S/)', 'SOLICITUD DE POSTVENTA', 'MONTO DE\nREEMBOLSO\n(US$)', 'CANAL DE REEMBOLS

In [ ]:
# 8. Definir las columnas objetivo para limpiar filas en base a esto
columnas_objetivo = [
    'nombres\napellidos\n!',
    'pais_origen',
    'ciudad',
    'n°_\ntelefono\n)',
    'e-mail\n*@'
]

df_total = df_total.dropna(subset=columnas_objetivo, how='all')
df_total

,nombres\napellidos\n!,pais_origen,ciudad,n°_\ntelefono\n),e-mail\n*@,fecha_aceptac.\n?,precio_de_venta_total\n(us$),precio_de_venta_total\n(€),Año,Hoja_Origen,fecha_de_aceptacion_del_viaje\n?,fecha_de_venta.\n?
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,06/03/23,5050.0,4848,2023,EDERLY,NaN,NaN
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,30/03/23,16640.0,15974.4,2023,EDERLY,NaN,NaN
6,Romain Lechifflart,Francia,NaN,33674347951,romain.lechifflart@gmail.com,25/04/23,1294.0,1216.36,2023,EDERLY,NaN,NaN
8,Benoit Trochu,Francia,NaN,33634661056,-,15/05/23,24874.0,23132.82,2023,EDERLY,NaN,NaN
10,Killian BRISSET,Francia,NaN,-,kilian.brisset@orange.f,05/06/23,432.0,410.4,2023,EDERLY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2984,LUCELLY SORIA AHUMADA,MEXICO,SINALOA,526671039510,NaN,3/7/2024,3915.0,NaN,2025,Yashira,NaN,NaN
2985,ANGELICA IVETTE RAMOS RODRIGUEZ,PUERTO RICO,PUERTO RICO,19392109372,NaN,3/6/2024,5800.0,NaN,2025,Yashira,NaN,NaN
2986,CONCEPCION Martinez DELGADO,ESPAÑA,CARTAGENA,34660843853,NaN,3/21/2024,5540.0,NaN,2025,Yashira,NaN,NaN
2987,JOEL MONTIJO RUIZ,USA,NEW YORK,+1 (787) 378-9600,NaN,3/23/2024,4600.0,NaN,2025,Yashira,NaN,NaN


arreglar fechas 

In [ ]:
# 9. Arreglar el formato de fechas de las 3 columnas

# Trabaja SIEMPRE sobre una copia materializada para evitar SettingWithCopyWarning
DF = df_total.copy()

# Columnas tal como existen ahora en tu DF (incluye saltos de línea si están)
cols_fecha = [
    "fecha_de_aceptacion_del_viaje\n?",
    "fecha_de_venta.\n?",
    "fecha_aceptac.\n?",
]

def parse_fecha_mixta_especifica(s: pd.Series) -> pd.Series:
    """Parsea probando formatos concretos para evitar el warning de inferencia.
       Orden: dd/mm/yyyy, dd/mm/yy, mm/dd/yyyy, yyyy-mm-dd.
    """
    s = s.astype(str).str.strip().str.replace("\n", " ", regex=False).str.lower()
    s = s.replace({"": np.nan, "nan": np.nan})

    out = pd.to_datetime(s, format="%d/%m/%Y", errors="coerce")

    m = out.isna() & s.notna()
    out.loc[m] = pd.to_datetime(s[m], format="%d/%m/%y", errors="coerce")

    m = out.isna() & s.notna()
    out.loc[m] = pd.to_datetime(s[m], format="%m/%d/%Y", errors="coerce")

    m = out.isna() & s.notna()
    out.loc[m] = pd.to_datetime(s[m], format="%Y-%m-%d", errors="coerce")

    return out

for col in cols_fecha:
    if col not in DF.columns:
        print(f"⚠️ Columna no encontrada: {repr(col)}")
        continue

    # Respaldo de la columna original (usa .loc para evitar el warning)
    DF.loc[:, f"{col}_raw"] = DF[col]

    # Parseo (sin mapear 'hoy/ayer')
    DF.loc[:, f"{col}_dt"] = parse_fecha_mixta_especifica(DF[col])

    # Mini-reporte
    tot = len(DF)
    ok = DF[f"{col}_dt"].notna().sum()
    print(f"✅ {repr(col)} → parseadas {ok}/{tot} | no convertidas: {tot-ok}")

# Si quieres que el resultado reemplace a df_total:
df_total = DF


✅ 'fecha_de_aceptacion_del_viaje\n?' → parseadas 31/1502 | no convertidas: 1471
✅ 'fecha_de_venta.\n?' → parseadas 60/1502 | no convertidas: 1442
✅ 'fecha_aceptac.\n?' → parseadas 1126/1502 | no convertidas: 376


In [22]:
df_total

,nombres\napellidos\n!,pais_origen,ciudad,n°_\ntelefono\n),e-mail\n*@,fecha_aceptac.\n?,Año,Hoja_Origen,fecha_de_aceptacion_del_viaje\n?,fecha_de_venta.\n?,fecha_de_aceptacion_del_viaje\n?_raw,fecha_de_aceptacion_del_viaje\n?_dt,fecha_de_venta.\n?_raw,fecha_de_venta.\n?_dt,fecha_aceptac.\n?_raw,fecha_aceptac.\n?_dt
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,06/03/23,2023,EDERLY,NaN,NaN,NaN,NaT,NaN,NaT,06/03/23,2023-03-06
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,30/03/23,2023,EDERLY,NaN,NaN,NaN,NaT,NaN,NaT,30/03/23,2023-03-30
6,Romain Lechifflart,Francia,NaN,33674347951,romain.lechifflart@gmail.com,25/04/23,2023,EDERLY,NaN,NaN,NaN,NaT,NaN,NaT,25/04/23,2023-04-25
8,Benoit Trochu,Francia,NaN,33634661056,-,15/05/23,2023,EDERLY,NaN,NaN,NaN,NaT,NaN,NaT,15/05/23,2023-05-15
10,Killian BRISSET,Francia,NaN,-,kilian.brisset@orange.f,05/06/23,2023,EDERLY,NaN,NaN,NaN,NaT,NaN,NaT,05/06/23,2023-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,LUCELLY SORIA AHUMADA,MEXICO,SINALOA,526671039510,NaN,3/7/2024,2025,Yashira,NaN,NaN,NaN,NaT,NaN,NaT,3/7/2024,2024-07-03
2985,ANGELICA IVETTE RAMOS RODRIGUEZ,PUERTO RICO,PUERTO RICO,19392109372,NaN,3/6/2024,2025,Yashira,NaN,NaN,NaN,NaT,NaN,NaT,3/6/2024,2024-06-03
2986,CONCEPCION Martinez DELGADO,ESPAÑA,CARTAGENA,34660843853,NaN,3/21/2024,2025,Yashira,NaN,NaN,NaN,NaT,NaN,NaT,3/21/2024,2024-03-21
2987,JOEL MONTIJO RUIZ,USA,NEW YORK,+1 (787) 378-9600,NaN,3/23/2024,2025,Yashira,NaN,NaN,NaN,NaT,NaN,NaT,3/23/2024,2024-03-23


In [ ]:
# 10. Seleccionar columnas con el formato de fecha correcto
# Partimos de tu consolidado
DF = df_total.copy()

# 1) Para cada columna original que tenga su par *_dt, reemplaza sus valores por los de *_dt
for col in list(DF.columns):
    dt_col = f"{col}_dt"
    if dt_col in DF.columns:
        DF[col] = DF[dt_col]

# 2) Elimina columnas auxiliares *_raw y *_dt
aux_cols = [c for c in DF.columns if c.endswith("_raw") or c.endswith("_dt")]
df_nuevo = DF.drop(columns=aux_cols, errors="ignore")

# Listo: df_nuevo tiene todas las columnas y, en las de fecha, los valores ya parseados.


In [24]:
df_nuevo

,nombres\napellidos\n!,pais_origen,ciudad,n°_\ntelefono\n),e-mail\n*@,fecha_aceptac.\n?,Año,Hoja_Origen,fecha_de_aceptacion_del_viaje\n?,fecha_de_venta.\n?
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,2023-03-06,2023,EDERLY,NaT,NaT
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,2023-03-30,2023,EDERLY,NaT,NaT
6,Romain Lechifflart,Francia,NaN,33674347951,romain.lechifflart@gmail.com,2023-04-25,2023,EDERLY,NaT,NaT
8,Benoit Trochu,Francia,NaN,33634661056,-,2023-05-15,2023,EDERLY,NaT,NaT
10,Killian BRISSET,Francia,NaN,-,kilian.brisset@orange.f,2023-06-05,2023,EDERLY,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
2984,LUCELLY SORIA AHUMADA,MEXICO,SINALOA,526671039510,NaN,2024-07-03,2025,Yashira,NaT,NaT
2985,ANGELICA IVETTE RAMOS RODRIGUEZ,PUERTO RICO,PUERTO RICO,19392109372,NaN,2024-06-03,2025,Yashira,NaT,NaT
2986,CONCEPCION Martinez DELGADO,ESPAÑA,CARTAGENA,34660843853,NaN,2024-03-21,2025,Yashira,NaT,NaT
2987,JOEL MONTIJO RUIZ,USA,NEW YORK,+1 (787) 378-9600,NaN,2024-03-23,2025,Yashira,NaT,NaT


In [ ]:
# 11. Obtener una sola fecha para cada fila 
# Trabaja sobre tu DF final (cámbialo si usas otro nombre)
DF = df_nuevo  # o df_nuevo si ya limpiaste auxiliares

cols_fecha = [
    "fecha_aceptac.\n?",
    "fecha_de_aceptacion_del_viaje\n?",
    "fecha_de_venta.\n?",
]

# Asegura que solo use las que existen
cols_fecha = [c for c in cols_fecha if c in DF.columns]
if not cols_fecha:
    raise ValueError("Ninguna de las columnas de fecha fue encontrada en el DataFrame.")

# Convierte a datetime (no rompe si ya lo son)
for c in cols_fecha:
    DF[c] = pd.to_datetime(DF[c], errors="coerce", dayfirst=True)

# Crea la columna 'date' con la fecha mínima por fila
DF["date"] = DF[cols_fecha].min(axis=1, skipna=True).dt.normalize()  # yyyy-mm-dd (00:00)

# (Opcional) Si quieres saber de cuál columna salió la mínima:
# stack = DF[cols_fecha].stack(dropna=True)
# idxmin = stack.groupby(level=0).idxmin()
# DF["date_source"] = idxmin.reindex(DF.index)

# Mini-reporte (opcional)
total = len(DF)
ok = DF["date"].notna().sum()
print(f"✅ 'date' rellenada en {ok}/{total} filas | vacío en {total-ok}")


✅ 'date' rellenada en 1217/1502 filas | vacío en 285


In [34]:
df_nuevo

,nombres\napellidos\n!,pais_origen,ciudad,n°_\ntelefono\n),e-mail\n*@,fecha_aceptac.\n?,precio_de_venta_total\n(us$),precio_de_venta_total\n(€),Año,Hoja_Origen,fecha_de_aceptacion_del_viaje\n?,fecha_de_venta.\n?,date
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,2023-03-06,5050.0,4848,2023,EDERLY,NaT,NaT,2023-03-06
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,2023-03-30,16640.0,15974.4,2023,EDERLY,NaT,NaT,2023-03-30
6,Romain Lechifflart,Francia,NaN,33674347951,romain.lechifflart@gmail.com,2023-04-25,1294.0,1216.36,2023,EDERLY,NaT,NaT,2023-04-25
8,Benoit Trochu,Francia,NaN,33634661056,-,2023-05-15,24874.0,23132.82,2023,EDERLY,NaT,NaT,2023-05-15
10,Killian BRISSET,Francia,NaN,-,kilian.brisset@orange.f,2023-06-05,432.0,410.4,2023,EDERLY,NaT,NaT,2023-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,LUCELLY SORIA AHUMADA,MEXICO,SINALOA,526671039510,NaN,2024-07-03,3915.0,NaN,2025,Yashira,NaT,NaT,2024-07-03
2985,ANGELICA IVETTE RAMOS RODRIGUEZ,PUERTO RICO,PUERTO RICO,19392109372,NaN,2024-06-03,5800.0,NaN,2025,Yashira,NaT,NaT,2024-06-03
2986,CONCEPCION Martinez DELGADO,ESPAÑA,CARTAGENA,34660843853,NaN,2024-03-21,5540.0,NaN,2025,Yashira,NaT,NaT,2024-03-21
2987,JOEL MONTIJO RUIZ,USA,NEW YORK,+1 (787) 378-9600,NaN,2024-03-23,4600.0,NaN,2025,Yashira,NaT,NaT,2024-03-23


In [ ]:
# 12. Normalizar los nombres de los paises 

# ========= 1) Seleccionar la columna de país en df_nuevo =========
if "pais_origen" in df_nuevo.columns:
    col_pais = "pais_origen"
elif "pais" in df_nuevo.columns:
    col_pais = "pais"
else:
    raise KeyError("No encuentro la columna de país. Usa 'pais_origen' o 'pais' en df_nuevo.")

# ========= 2) Helpers =========
def _norm_txt(x: str) -> str:
    s = str(x).strip().lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode()
    s = re.sub(r"\s+", " ", s)
    return s

# ========= 3) Variantes -> país canónico (solo los de tu lista) =========
VAR_TO_CANON = {
    # Alemania
    "alemania": "Alemania",
    # Argentina
    "argentina": "Argentina",
    # Austria
    "austria": "Austria",
    # Bélgica
    "belgica": "Bélgica", "belgique": "Bélgica", "belgium": "Bélgica", "be": "Bélgica",
    # Brasil
    "brasil": "Brasil", "brazil": "Brasil", "nrasil": "Brasil",
    # Canadá
    "canada": "Canadá", "can": "Canadá",
    # Chile
    "chile": "Chile",
    # China
    "china": "China",
    # Colombia
    "colombia": "Colombia", "col": "Colombia",
    # Costa Rica
    "costa rica": "Costa Rica", "cr": "Costa Rica",
    # Rep. Dominicana
    "republica dominicana": "República Dominicana", "rd": "República Dominicana", "dominicana": "República Dominicana",
    # Ecuador
    "ecuador": "Ecuador",
    # El Salvador
    "el salvador": "El Salvador",
    # España
    "espana": "España", "esp": "España", "espanha": "España",
    # Estados Unidos
    "estados unidos": "Estados Unidos", "eua": "Estados Unidos", "eeuu": "Estados Unidos",
    "united states": "Estados Unidos", "usa": "Estados Unidos", "usa (puerto rico)": "Estados Unidos",
    # Finlandia
    "finlandia": "Finlandia",
    # Francia
    "francia": "Francia", "france": "Francia", "fr": "Francia", "fra": "Francia", "frs": "Francia",
    # Grecia
    "grecia": "Grecia",
    # Guatemala
    "guatemala": "Guatemala",
    # Países Bajos
    "holanda": "Países Bajos", "netherlands": "Países Bajos", "netherland": "Países Bajos",
    # Italia
    "italia": "Italia",
    # Corea
    "corea": "Corea", "korea": "Corea",
    # México
    "mexico": "México", "mexco": "México", "mx": "México", "mexc": "México", "méxico": "México",
    # Nicaragua
    "nicaragua": "Nicaragua",
    # Panamá
    "panama": "Panamá", "panamá": "Panamá",
    # Perú
    "peru": "Perú", "perú": "Perú",
    # Portugal
    "portugal": "Portugal",
    # Puerto Rico
    "puerto rico": "Puerto Rico", "pr": "Puerto Rico",
    # Rusia
    "rusia": "Rusia",
    # Suiza
    "suiza": "Suiza", "suisse": "Suiza", "suizo": "Suiza",
    # Tailandia
    "thailand": "Tailandia",
    # Uruguay
    "uruguay": "Uruguay",
    # Surinam
    "surinam": "Surinam", "suriname": "Surinam",

    # --- Variantes NO válidas (ciudades, combos, personas) -> descartar (None)
    "miami": None,
    "colombia y peru": None, "netherlands / suriname": None, "francia y italia": None,
    "thailand / netherland": None, "thailand / netherlands": None,
    "espanol": None, "espanol ?": None, "espanol-": None, "espanol/": None,  # idioma, no país
    "joselyn fallas zuniga": None,  # persona
}

ALLOWED = {
    "Alemania","Argentina","Austria","Bélgica","Brasil","Canadá","Chile","China","Colombia",
    "Costa Rica","República Dominicana","Ecuador","El Salvador","España","Estados Unidos",
    "Finlandia","Francia","Grecia","Guatemala","Países Bajos","Italia","Corea","México",
    "Nicaragua","Panamá","Perú","Portugal","Puerto Rico","Rusia","Suiza","Tailandia","Uruguay","Surinam"
}

def normalize_country(value) -> str | None:
    key = _norm_txt(value)
    canon = VAR_TO_CANON.get(key)
    if canon is None:           # no mapeado o marcado para excluir
        return None
    if canon not in ALLOWED:    # no está en la lista estricta
        return None
    return canon

# ========= 4) Aplicar en df_nuevo =========
df_nuevo["pais_norm"] = df_nuevo[col_pais].map(normalize_country)

# ==============================
# 🚧 FILTRO ESTRICTO AQUÍ:
#    Mantén SOLO filas con país permitido (resto: eliminar).
# ==============================
antes = len(df_nuevo)
df_nuevo = df_nuevo[df_nuevo["pais_norm"].notna()].copy()
despues = len(df_nuevo)
print(f"Filas mantenidas: {despues} / {antes}  |  Eliminadas: {antes - despues}")

# (Opcional) reemplazar columna original por la normalizada y limpiar aux
# df_nuevo[col_pais] = df_nuevo["pais_norm"]
# df_nuevo.drop(columns=["pais_norm"], inplace=True)


Filas mantenidas: 1427 / 1502  |  Eliminadas: 75


In [37]:
df_nuevo

,nombres_apellidos,pais,ciudad,telefono,email,fecha_aceptacion,precio_venta_usd,precio_venta_eur,año,hoja_origen,fecha_aceptacion_viaje,fecha_venta,date,pais_norm
3,Pascale Bousquet,Francia,NaN,33608465939,pascale.bousquet.pouget@orange.fr,2023-03-06,5050.0,4848,2023,EDERLY,NaT,NaT,2023-03-06,Francia
4,Annie Anjollini,Francia,NaN,33676854104,annie.anjollini@noos.fr,2023-03-30,16640.0,15974.4,2023,EDERLY,NaT,NaT,2023-03-30,Francia
6,Romain Lechifflart,Francia,NaN,33674347951,romain.lechifflart@gmail.com,2023-04-25,1294.0,1216.36,2023,EDERLY,NaT,NaT,2023-04-25,Francia
8,Benoit Trochu,Francia,NaN,33634661056,-,2023-05-15,24874.0,23132.82,2023,EDERLY,NaT,NaT,2023-05-15,Francia
10,Killian BRISSET,Francia,NaN,-,kilian.brisset@orange.f,2023-06-05,432.0,410.4,2023,EDERLY,NaT,NaT,2023-06-05,Francia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,LUCELLY SORIA AHUMADA,MEXICO,SINALOA,526671039510,NaN,2024-07-03,3915.0,NaN,2025,Yashira,NaT,NaT,2024-07-03,México
2985,ANGELICA IVETTE RAMOS RODRIGUEZ,PUERTO RICO,PUERTO RICO,19392109372,NaN,2024-06-03,5800.0,NaN,2025,Yashira,NaT,NaT,2024-06-03,Puerto Rico
2986,CONCEPCION Martinez DELGADO,ESPAÑA,CARTAGENA,34660843853,NaN,2024-03-21,5540.0,NaN,2025,Yashira,NaT,NaT,2024-03-21,España
2987,JOEL MONTIJO RUIZ,USA,NEW YORK,+1 (787) 378-9600,NaN,2024-03-23,4600.0,NaN,2025,Yashira,NaT,NaT,2024-03-23,Estados Unidos


In [ ]:
# 13. Preparar el df_nuevo para exportarlo a excel
# 1) Eliminar la columna "Vendedor"
df_nuevo = df_nuevo.drop(columns=["Vendedor"], errors="ignore")  # no falla si no existe
mapeo_renombres = {
    "nombres\napellidos\n!": "nombres_apellidos",
    "pais_origen": "pais",
    "ciudad": "ciudad",
    "n°_\ntelefono\n)": "telefono",
    "e-mail\n*@": "email",
    "Año": "año",
    "Hoja_Origen": "hoja_origen",
    "Archivo_Origen": "archivo_origen",
    "fecha_aceptac.\n?":"fecha_aceptacion",
    "fecha_de_aceptacion_del_viaje\n?":"fecha_aceptacion_viaje",
    "fecha_de_venta.\n?": "fecha_venta",
    "precio_de_venta_total\n(us$)": "precio_venta_usd",
    "precio_de_venta_total\n(€)" : "precio_venta_eur"
}
df_nuevo = df_nuevo.rename(columns=mapeo_renombres)
# 3) Exportar a Excel (sin índice)
ruta_salida = r"C:\Users\Lima - Rodrigo\Documents\ventas\bd\bd_fecha_precio.xlsx"
df_nuevo.to_excel(ruta_salida, index=False, sheet_name="Consolidado")  # requiere openpyxl instalado

print(f"✅ Archivo exportado a: {ruta_salida}")

✅ Archivo exportado a: C:\Users\Lima - Rodrigo\Documents\ventas\bd\bd_fecha_precio.xlsx
